# Demo T1 processing notebook

## Startup qMRLab

In [1]:
try
    cd qMRLab
    startup
    cd ..
catch
    error("qMRLab could not be started correctly.")
end

## Get configuration options



In [2]:
datasetDetails=loadjson('example.json');
submission = fieldnames(datasetDetails);

dataTypeList = {};
imagePathList = {};
counter = 1;

for ii = 1:length(submission)
    datasetList = fieldnames(datasetDetails.(submission{ii}).datasets);

    for jj = 1:length(datasetList)
        OSF_linkList{counter} = datasetDetails.(submission{ii}).OSF_link;

        siteDetails=datasetDetails.(submission{ii}).datasets.(datasetList{jj});
        dataTypeList{counter} = siteDetails.dataType;
        imagePathList{counter} = siteDetails.imagePath;
        counter = counter + 1;
    end
end

## Download data

In [3]:
downloadedList = {''}
counter = 1;

for ii = 1:length(imagePathList)

    OSF_link= OSF_linkList{ii};

    if ~strcmp(OSF_link, downloadedList)
        % Download data
        download_data(OSF_link);
        
        downloadedList{counter} = OSF_link;
        counter = counter + 1;
    end
    
end


downloadedList =

  1x1 cell array

    {0x0 char}

Dataset downloaded succesfully!


## Convert YAML config files to YAML

In [4]:
%get dataTypeList --from MATLAB
%get imagePathList --from MATLAB

from convert_yaml_to_json import *

for dataType, imagePath in zip(dataTypeList, imagePathList):
   convert_yaml_to_json(dataType, imagePath)


Converting YAML file 20200227_karakuzu_polymtl_NIST/20191213_UNF/20191213_karakuzu_UNF_NIST_FS.nii.gz to JSON.
Data type is: Magnitude
Conversion complete!
Converting YAML file 20200227_karakuzu_polymtl_NIST/20200227_MNI/20200227_karakuzu_MNI_NIST_FS.nii.gz to JSON.
Data type is: Magnitude
Conversion complete!


## Process entire dataset

In [8]:
for ii = 1:length(imagePathList)

    dataType = dataTypeList{ii};
    imagePath = imagePathList{ii};


    % Load data
    [data, hdr] = load_data(dataType, imagePath);

    % Get inversion times
    TI = load_inversion_times(imagePath);

    % Fit data
    FitResults = fit_inversion_recovery(data, TI, dataType);

    % Save data to nifti

    [filepath,name,ext] = fileparts(imagePath);
    if strcmp(name(end-3:end),'.nii')
       [tmp1,name,tmp2] = fileparts(name);
    end
    clear tmp1, tmp2;

    t1MapPath = ['t1_map' filesep filepath];
    t1Filename = [name '_t1map.nii.gz'];

    save_to_nifti(FitResults, hdr, t1MapPath, t1Filename)
    
end

Loading data of type: Magnitude
Data loaded succesfully!
Loading inversion times...
Inversion times for data/20200227_karakuzu_polymtl_NIST/20191213_UNF/20191213_karakuzu_UNF_NIST_FS.json loaded succefully!
Starting to fit data...
=============== qMRLab::Fit ======================
Operation has been started: inversion_recovery
Elapsed time is 60.469648 seconds.
Operation has been completed: inversion_recovery
Data fit succesfully!
Saving T1 map to NIFTI...
Save complete!
Loading data of type: Magnitude
Data loaded succesfully!
Loading inversion times...
Inversion times for data/20200227_karakuzu_polymtl_NIST/20200227_MNI/20200227_karakuzu_MNI_NIST_FS.json loaded succefully!
Starting to fit data...
=============== qMRLab::Fit ======================
Operation has been started: inversion_recovery
Elapsed time is 84.081344 seconds.
Operation has been completed: inversion_recovery
Data fit succesfully!
Saving T1 map to NIFTI...
Save complete!
